<a href="https://colab.research.google.com/github/madhuselvaraj/tensorflow_projects/blob/master/jigja_toxic_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd
import numpy as np

In [0]:
os.mkdir("/content/Dataset")

In [0]:
train_data = pd.read_csv("/content/Dataset/train.csv")

In [0]:
train_data.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [0]:
train_data.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [0]:
train_classes = ["toxic","severe_toxic"	,"obscene"	, "threat", "insult",	"identity_hate"]
Y = train_data[train_classes].values
print(Y)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [0]:
train_sentence = train_data["comment_text"]
print(train_sentence[:3])

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
Name: comment_text, dtype: object


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input,Embedding,LSTM,Dense,Dropout,Activation
from keras.models import Model

In [0]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sentence))
trained_tokenizer = tokenizer.texts_to_sequences(train_sentence)

In [0]:
print(trained_tokenizer[0])

[688, 75, 1, 126, 130, 177, 29, 672, 4511, 12052, 1116, 86, 331, 51, 2278, 11448, 50, 6864, 15, 60, 2756, 148, 7, 2937, 34, 117, 1221, 15190, 2825, 4, 45, 59, 244, 1, 365, 31, 1, 38, 27, 143, 73, 3462, 89, 3085, 4583, 2273, 985]


In [0]:
maxlen = 200
X = pad_sequences(trained_tokenizer, maxlen=maxlen)

In [0]:
def model(input_shape,embedded_layer_shape):
  
  X_input = Input(shape = (input_shape,)) 
  X = Embedding(max_features,128)(X_input)
  X = LSTM(60, return_sequences=True,name='lstm_layer')(X)
  X = GlobalMaxPooling1D()(X)
  X = Dropout(0.4)(X)
  X = Dense(50, activation="relu")(X)
  X = Dropout(0.3)(X)
  X = Dense(6, activation="sigmoid")(X)
  
  model = Model(inputs=X_input, outputs=X)
  
  return model

In [0]:
Model = model(input_shape = maxlen,embedded_layer_shape =  max_features)
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
lstm_layer (LSTM)            (None, 200, 60)           45360     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 60)                0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 60)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 50)                3050      
_________________________________________________________________
dropout_16 (Dropout)         (None, 50)                0         
__________

In [0]:
def new_model(input_shape,embedded_layer_shape):
  
  X_input = Input(shape = (input_shape,)) 
  X = Embedding(embedded_layer_shape,128)(X_input)
  X = LSTM(128, return_sequences=True)(X)
  X = Dropout(0.5)(X)
  X = LSTM(128)(X)
  X = Dropout(0.5)(X)
  X = Dense(6, activation="sigmoid")(X)
  #X = Activation(('softmax'))(X)
  
  model = Model(inputs=X_input, outputs=X)
  
  return model

In [0]:
Model = new_model(input_shape = maxlen,embedded_layer_shape =  max_features)
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 200)               0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 200, 128)          2560000   
_________________________________________________________________
lstm_15 (LSTM)               (None, 200, 128)          131584    
_________________________________________________________________
dropout_29 (Dropout)         (None, 200, 128)          0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 774       
Total para

In [0]:
Model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [0]:
Model.fit(X,Y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1337s 9ms/step - loss: 0.0728 - acc: 0.9770 - val_loss: 0.0511 - val_acc: 0.9817
Epoch 2/2
143613/143613 [==============================] - 1339s 9ms/step - loss: 0.0483 - acc: 0.9824 - val_loss: 0.0494 - val_acc: 0.9820


In [0]:
Model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
Model.fit(X,Y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 2629s 18ms/step - loss: 0.0694 - acc: 0.9778 - val_loss: 0.0513 - val_acc: 0.9815
Epoch 2/2
143613/143613 [==============================] - 2591s 18ms/step - loss: 0.0474 - acc: 0.9825 - val_loss: 0.0488 - val_acc: 0.9822


In [0]:
test_comment = ["Catalan independentism is the social movement involving more people in Catalonia. Votes in the referendum at Barcelona were more than the ones received by political parties that are leading the council (PSC and CIU) . 20% of votes of the total census voting in favour of a referendum that is forbidden by spanish courts and that has no legal effects is highly relevant.","COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK","her"]
test_tokenizer = tokenizer.texts_to_sequences(test_comment)
test_X = pad_sequences(test_tokenizer, maxlen=maxlen)

In [0]:
print(test_X)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0  8304     8     1
   1179  1190  3172    57    74    10 15051  2075    10     1 10114    34
   8397    86    57    93     1   779 

In [0]:
output = Model.predict_on_batch(test_X)
print(output)

[[8.6694956e-05 0.0000000e+00 3.9041042e-06 0.0000000e+00 2.8908253e-06
  2.6822090e-07]
 [9.9900866e-01 3.6265054e-01 9.7617096e-01 3.4475803e-02 8.5658234e-01
  1.2317154e-01]
 [9.5839202e-03 5.4031610e-05 9.5787644e-04 1.8304586e-04 1.1480717e-03
  4.2011519e-04]]


In [0]:
output = Model.predict_on_batch(test_X)
print(output)

[[9.8371506e-04 1.5199184e-06 8.1837177e-05 1.4901161e-06 1.5527010e-04
  3.7223101e-05]
 [9.9864602e-01 4.1532347e-01 9.8516357e-01 5.9445977e-02 8.8768256e-01
  1.6295904e-01]
 [2.1024814e-01 1.0987818e-03 2.5021791e-02 3.4689605e-03 5.3496558e-02
  9.4942376e-03]]
